Idea is to read in the annotation file of emapper.py and pull out the annotations.
This was run like:
/home/benjamin/anaconda3/envs/funannotate/eggnog-mapper-0.99.2/emapper.py -i /home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/032017_assembly/Pst_104E_v12_h_ctg.anno.protein.fa -d euk --output Pst_104E_v12_h_ctg --cpu 12


In [1]:
import pandas as pd
import os
import re
from Bio import SeqIO
import pysam
from Bio.SeqRecord import SeqRecord
from pybedtools import BedTool
import numpy as np
import pybedtools
import time
import matplotlib.pyplot as plt
import sys
import subprocess
import shutil

In [63]:
BASE_FOLDER = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/protein_annotation/Pst_104E_v12_h_ctg/eggnog-mapper'
EGGNOG_BLAST_FILE = 'Pst_104E_v12_h_ctg.emapper.annotations'
EGGNOG_DIAMON_FILE = 'Pst_104E_v12_h_ctg_diamond.emapper.annotations'
BASE_AA_PATH = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12'
BASE_A_PATH = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/032017_assembly'
p_genome = 'Pst_104E_v12_h_ctg'
OUT_PATH = os.path.join(BASE_AA_PATH, 'protein_annotation',p_genome, 'eggnog-mapper', 'parsed')
if not os.path.exists(OUT_PATH):
    os.mkdir(OUT_PATH)


In [6]:
#pull out all proteins that are in the final assembly
p_protein_list = []
protein_fa_file = [x for x in os.listdir(BASE_A_PATH) if p_genome in x and x.endswith('anno.protein.fa')][0]
for protein in SeqIO.parse(os.path.join(BASE_A_PATH, protein_fa_file) , 'fasta'):
    p_protein_list.append(protein.id)

In [8]:
eggnog_blast_header = 'query_name seed_eggNOG_ortholog seed_ortholog_evalue seed_ortholog_score predicted_gene_name \
GO_terms KEGG_pathways Annotation_tax_scope OGs bestOG|evalue|score COG cat eggNOG annot'.split(' ')

In [9]:
eggnog_blast_header

['query_name',
 'seed_eggNOG_ortholog',
 'seed_ortholog_evalue',
 'seed_ortholog_score',
 'predicted_gene_name',
 'GO_terms',
 'KEGG_pathways',
 'Annotation_tax_scope',
 'OGs',
 'bestOG|evalue|score',
 'COG',
 'cat',
 'eggNOG',
 'annot']

In [62]:
eggnog_blast_df = pd.read_csv(os.path.join(BASE_FOLDER, EGGNOG_BLAST_FILE), sep ='\t', header=None, names=eggnog_blast_header, skiprows=3)

FileNotFoundError: File b'/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/protein_annotation/Pst_104E_v12_h_ctg/eggnog-mapper/Pst_104E_v12_h_ctg.emapper.annotations' does not exist

In [21]:
eggnog_blast_df.iloc[1,:]

query_name                                   evm.model.hcontig_010_016.50
seed_eggNOG_ortholog                                        5297.EFP85053
seed_ortholog_evalue                                                    0
seed_ortholog_score                                                  1155
predicted_gene_name                                             FG02823.1
GO_terms                GO:0003674,GO:0003824,GO:0004620,GO:0004622,GO...
KEGG_pathways                                                    map00564
Annotation_tax_scope                                            euNOG[57]
OGs                                               0XR72@NOG,KOG1325@euNOG
bestOG|evalue|score                                KOG1325|2.4e-110|374.0
COG                                                                     I
cat                                                phospholipase A2 group
eggNOG                                                                  0
annot                                 

In [52]:
eggnog_blast_df.fillna(0, inplace =True)

In [24]:
eggnog_blast_df.columns

Index(['query_name', 'seed_eggNOG_ortholog', 'seed_ortholog_evalue',
       'seed_ortholog_score', 'predicted_gene_name', 'GO_terms',
       'KEGG_pathways', 'Annotation_tax_scope', 'OGs', 'bestOG|evalue|score',
       'COG', 'cat', 'eggNOG', 'annot'],
      dtype='object')

In [34]:
DBs

['predicted_gene_name',
 'GO_terms',
 'KEGG_pathways',
 'Annotation_tax_scope',
 'OGs',
 'COG',
 'cat',
 'eggNOG',
 'annot']

In [45]:
#pick all annotation columns. One at a time and write them out as tab file or annotations files. The later can be used
#to annotate gff files using gag.py
DBs = [x for x in eggnog_blast_df.columns.tolist()[4:] if x not in ['GO_terms','KEGG_pathways', 'OGs' ] ]
eggnog_blast_df['note'] = 'note'
for db in DBs:
    if len(eggnog_blast_df[eggnog_blast_df[db] !=0]) > 0:
        eggnog_blast_df[eggnog_blast_df[db] !=0].loc[:,['query_name',db]]\
        .to_csv(os.path.join(OUT_PATH, db+'_terms.tab'), sep='\t', header =None, index = None)
        eggnog_blast_df[eggnog_blast_df[db] !=0].loc[:,['query_name', 'note', db]]\
        .to_csv(os.path.join(OUT_PATH, 'annotations.' +db+'.txt'), sep='\t', header =None, index = None)
    
    

In [60]:
working_dict = {}
eggnog_blast_df['GO_terms'] = eggnog_blast_df.GO_terms.str.split(',')

eggnog_blast_df[eggnog_blast_df['GO_terms'] != 'NaN']

AttributeError: Can only use .str accessor with string values, which use np.object_ dtype in pandas

In [61]:
eggnog_blast_df['GO_terms']

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
5      NaN
6      NaN
7      NaN
8      NaN
9      NaN
10     NaN
11     NaN
12     NaN
13     NaN
14     NaN
15     NaN
16     NaN
17     NaN
18     NaN
19     NaN
20     NaN
21     NaN
22     NaN
23     NaN
24     NaN
25     NaN
26     NaN
27     NaN
28     NaN
29     NaN
        ..
6892   NaN
6893   NaN
6894   NaN
6895   NaN
6896   NaN
6897   NaN
6898   NaN
6899   NaN
6900   NaN
6901   NaN
6902   NaN
6903   NaN
6904   NaN
6905   NaN
6906   NaN
6907   NaN
6908   NaN
6909   NaN
6910   NaN
6911   NaN
6912   NaN
6913   NaN
6914   NaN
6915   NaN
6916   NaN
6917   NaN
6918   NaN
6919   NaN
6920   NaN
6921   NaN
Name: GO_terms, Length: 6922, dtype: float64

In [ ]:
#process the pathway files
interpro_by_protein_KEGG = interpro_df[interpro_df.Pathway_IDs.str.contains('KEGG' or 'MetaCyc' or 'Reactome').fillna(False)]

interpro_by_protein_KEGG = interpro_by_protein_KEGG.groupby('Updated_Protein_ID')

#pull out all the KEGG terms and write them out as annotation files 
interpro_by_protein_KEGG = interpro_by_protein.Pathway_IDs.apply(set)

#remove everything without KEGG term attached
interpro_by_protein_KEGG = interpro_by_protein_KEGG[~(interpro_by_protein_KEGG == {0})]

interpro_by_protein_KEGG_dict = dict(zip(interpro_by_protein_KEGG.index, interpro_by_protein_KEGG))

ALL_KEGG_LIST = []
ALL_PROTEIN_INDEX_LIST = []
for key in list(interpro_by_protein_KEGG_dict.keys()):
    _tmp_list = list(interpro_by_protein_KEGG_dict[key])
    #remove 0 
    _tmp_list = [x for x in _tmp_list if x != 0]
    new_value = []
    for x in _tmp_list: 
        if '|' in x:
            _list = x.split('|')
            for y in _list:
                new_value.append(y)
        else:
            new_value.append(x)
    new_value = list(set(new_value))
    new_index = [key]*len(new_value)
    ALL_KEGG_LIST += new_value
    ALL_PROTEIN_INDEX_LIST += new_index


KEGG_df = pd.DataFrame([ALL_PROTEIN_INDEX_LIST, ALL_KEGG_LIST]).T
KEGG_df.rename(columns={0:'Updated_Protein_ID', 1:'DB_ID'}, inplace=True)

KEGG_df.to_csv(os.path.join(OUT_PATH_P_ALL, 'Pathway_terms_ipr_all.tab') , sep = '\t', header =None, index=None)
KEGG_df[KEGG_df.Updated_Protein_ID.isin(p_protein_list)].to_csv(os.path.join(OUT_PATH_P, 'Pathway_terms_ipr.tab') , sep = '\t', header =None, index=None)
#write out annotations
KEGG_df['Transfer_ID'] = 'note'
KEGG_df.loc[:,['Updated_Protein_ID', 'Transfer_ID','DB_ID']].to_csv(os.path.join(OUT_PATH_P_ALL, 'annotations.Pathway_all.txt') , sep = '\t', header =None, index=None)
KEGG_df[KEGG_df.Updated_Protein_ID.isin(p_protein_list)].loc[:,['Updated_Protein_ID', 'Transfer_ID','DB_ID']].to_csv(os.path.join(OUT_PATH_P, 'annotations.Pathway.txt') , sep = '\t', header =None, index=None)

In [ ]:
#pull out all the GO terms and write them out as annotation files 
interpro_by_protein_GO = interpro_by_protein.GO_terms.apply(set)

#remove everything without GO term attached
interpro_by_protein_GO = interpro_by_protein_GO[~(interpro_by_protein_GO == {0})]

interpro_by_protein_GO_dict = dict(zip(interpro_by_protein_GO.index, interpro_by_protein_GO))

ALL_GO_LIST = []
ALL_PROTEIN_INDEX_LIST = []
for key in list(interpro_by_protein_GO_dict.keys()):
    _tmp_list = list(interpro_by_protein_GO_dict[key])
    #remove 0 
    _tmp_list = [x for x in _tmp_list if x != 0]
    new_value = []
    for x in _tmp_list: 
        if '|' in x:
            _list = x.split('|')
            for y in _list:
                new_value.append(y)
        else:
            new_value.append(x)
    new_value = list(set(new_value))
    new_index = [key]*len(new_value)
    ALL_GO_LIST += new_value
    ALL_PROTEIN_INDEX_LIST += new_index


GO_df = pd.DataFrame([ALL_PROTEIN_INDEX_LIST, ALL_GO_LIST]).T
GO_df.rename(columns={0:'Updated_Protein_ID', 1:'DB_ID'}, inplace=True)

GO_df.to_csv(os.path.join(OUT_PATH_P_ALL, 'GO_terms_ipr_all.tab') , sep = '\t', header =None, index=None)
GO_df[GO_df.Updated_Protein_ID.isin(p_protein_list)].to_csv(os.path.join(OUT_PATH_P, 'GO_terms_ipr.tab') , sep = '\t', header =None, index=None)
#write out annotations
GO_df['Transfer_ID'] = 'note'
GO_df.loc[:,['Updated_Protein_ID', 'Transfer_ID','DB_ID']].to_csv(os.path.join(OUT_PATH_P_ALL, 'annotations.GO_all.txt') , sep = '\t', header =None, index=None)
GO_df[GO_df.Updated_Protein_ID.isin(p_protein_list)].loc[:,['Updated_Protein_ID', 'Transfer_ID','DB_ID']].to_csv(os.path.join(OUT_PATH_P, 'annotations.GO.txt') , sep = '\t', header =None, index=None)

In [ ]:
#filter out other annotations files including busco, dbCAN, merops, swissprot
annotation_files = [os.path.join(ANNOTATION_FILE_BASE_PATH, x) for x in os.listdir(ANNOTATION_FILE_BASE_PATH) \
                    if x.startswith('annotations') and (x.endswith('busco.txt') or x.endswith('dbCAN.txt') or x.endswith('merops.txt') or x.endswith('swissprot.txt'))]

In [ ]:
for anno in annotation_files:
    _tmp_df = pd.read_csv(anno, header=None, sep='\t', names=['Protein_ID','Transfer_ID', 'DB_ID'])
    anno_file_name = anno.split('/')[-1]
    anno_midfix = anno_file_name.split('.')[1]
    _tmp_df['Updated_Protein_ID'] = protein_id_conversion(_tmp_df)
    #write out filtered down tab file
    _tmp_df[(_tmp_df.Updated_Protein_ID.isin(p_protein_list))].loc[:,['Updated_Protein_ID', 'DB_ID']].to_csv(os.path.join(OUT_PATH_P, anno_midfix+'_terms.tab'), sep = '\t', header =None, index=None)
    #write out the filtered annotation file
    _tmp_df[(_tmp_df.Updated_Protein_ID.isin(p_protein_list))].loc[:,['Updated_Protein_ID', 'Transfer_ID','DB_ID']].\
    to_csv(os.path.join(OUT_PATH_P, anno_file_name ), sep = '\t', header =None, index=None)
     #write out not down tab file
    _tmp_df.loc[:,['Updated_Protein_ID', 'DB_ID']].to_csv(os.path.join(OUT_PATH_P_ALL, anno_midfix+'_terms_all.tab' ), sep = '\t', header =None, index=None)
    #write out the filtered annotation file
    _tmp_df.loc[:,['Updated_Protein_ID', 'Transfer_ID','DB_ID']].to_csv(os.path.join(OUT_PATH_P_ALL, anno_file_name.replace('.txt', '_all.txt') ), sep = '\t', header =None, index=None)